# Introdução

Neste kernel usaremos python para programar um agente de *reinforcement learning* para solucionar o jogo de *black jack*, famoso "vinte e um". Nosso jogador será treinado em um ambiente simulado e aprenderá sozinho a vencer a banca (ou pelo menos tentar).

Esta aula é baseada no micro-desafio relacionado do [Kaggle](https://www.kaggle.com/fernandocanteruccio/blackjack-microchallenge-reinforce-agent).

![Blackjack](http://www.hightechgambling.com/sites/default/files/styles/large/public/casino/table_games/blackjack.jpg)

# Regras do Blackjack

Usaremos uma versão ligeiramente simplificada do blackjack (também conhecido como vinte e um). Nesta versão, há um jogador e um dealer. O jogo prossegue da seguinte forma:

- O jogador recebe duas cartas viradas para cima. O dealer recebe uma carta com a face para cima.
- O jogador pode pedir para receber outra carta ('pedir') quantas vezes quiser. Se a soma de suas cartas exceder 21, ele perde a rodada imediatamente.
- O *dealer* então distribui cartas adicionais para si mesmo até:
    - A soma das cartas do *dealer* exceder 21, nesse caso o jogador vence a rodada, ou
    - A soma das cartas do *dealer* ser maior ou igual a 17. Se o total do jogador for maior que o do *dealer*, o jogador vence. Caso contrário, o *dealer* vence (mesmo em caso de empate).

Ao calcular a soma de cartas, Valete, Dama e Rei contam como 10. Ases podem contar como 1 ou 11. (Quando se refere ao "total" de um jogador acima, queremos dizer o maior total que pode ser feito sem exceder 21. A + 8 = 19, A + 8 + 8 = 17.)

# A mesa

Certo, com o conhecimento das regras do jogo, podemos escrever um algoritmo básico para simular uma 'mesa'.

Vamos começar importando nossas dependências, neste caso, apenas o [Numpy](https://numpy.org/).

In [1]:
import numpy as np
from random import shuffle
np.random.seed(1)

Para jogar cartas precisamos de um baralho, certo? Vamos implementar um!

Utilizaremos um gerador para simular nosso baralho, assim poderemos tirar uma carta por vez mantendo um estado que nos indica quais cartas já foram utilizadas. Nosso baralho é descrito por uma lista de números de 1 a 11, sendo que teremos quatro 10 de cada nipe, representando o número 10 em si, além do Valete, da Rainha e do Rei.

Embaralhamos o baralho com a função shuffle antes de realizar a primeira compra.

In [2]:
# generator for card drawing without repetition
def get_deck():
    cards = (list(range(1, 9)) + [10] * 4 + [11]) * 4
    shuffle(cards)
    while cards:
        yield cards.pop()

Já podemos instanciar um baralho e utiliza-lo para comprar cartas.

In [3]:
deck = get_deck()
for i in range(2):
    print(next(deck))

7
10


O Vinte e um tem regras bem específicas. Precisamos calcular o valor total de uma mão dependendo do número de Ases presentes na mesma.

In [4]:
def calculate_total(aces, partial):
    total = partial
    for i in range(aces):
        if partial + 11 > 21:
            total += 1          # if score > 21, aces have value 1
        else:
            total += 11         # otherwise, aces have value 11
    return total

A representação de nossa mesa consiste no valor total e no número de Ases na mão de cada jogador.

In [5]:
def get_environment(dealer_partial, dealer_aces, player_partial, player_aces):
    return (calculate_total(player_aces, player_partial),
        calculate_total(dealer_aces, dealer_partial),
        player_aces)

Seguindo as regras descritas, podemos simular um jogo completo. Utilizamos variáveis de estado para manter os valores parciais e o número de Ases na mão de cada jogador. A cada rodada, atualizamos os valores destas variáveis para refletir as cartas sendo compradas.

No fim do jogo, retornamos 1 caso o jogador vença e 0 caso o jogador "estoure" o limite ou o *dealer* vença por pontos.

In [6]:
def simulate_game(agent):
    # init scores and deck
    dealer_partial = 0
    dealer_aces = 0
    player_partial = 0
    player_aces = 0
    deck = get_deck()
    
    # initial draw for the player
    for _ in range(2):
        card = next(deck)
        if card == 11:
            player_aces += 1
        else:
            player_partial += card
            
    # then for the dealer
    card = next(deck)
    if card == 11:
        dealer_aces += 1
    else:
        dealer_partial += card
    
    # player's turn
    # draw cards according to the provided policy
    while agent(*get_environment(dealer_partial, dealer_aces, player_partial, player_aces)):
        card = next(deck)
        if card == 11:
            player_aces += 1
            
        else:
            player_partial += card
            
        if calculate_total(player_aces, player_partial) > 21:
            return 0 # return 0 indicating house's victory
        
    # dealer's turn
    while calculate_total(dealer_aces, dealer_partial) < 17:
        card = next(deck)
        if card == 11:
            dealer_aces += 1
        else:
            dealer_partial += card
            
    # calculate totals
    player_total = calculate_total(player_aces, player_partial)
    dealer_total = calculate_total(dealer_aces, dealer_partial)
    
    # return 1 for player's victory, 0 otherwise
    if dealer_total > 21 or player_total > dealer_total:
        return 1
    return 0

Para simular vários jogos e nos dar estatísticas da qualidade do nosso jogador, utilizamos um loop simples.

In [7]:
def simulate(agent, n_games=5):
    player_victories = 0
    for i in range(n_games):
        player_victories += simulate_game(agent)
        
        if (i + 1) % 1000 == 0:
            print(f'{i}/{n_games} - Player won {player_victories} out of {i} (win rate = {player_victories / (i + 1) * 100}%)', end='\r')
        
    print(f'\nPlayer won {player_victories} out of {n_games} (win rate = {player_victories / n_games * 100}%)')

# O jogador de blackjack

Nosso jogador será representado pela classe *Agent*. A aplicação da instância deverá corresponder a estratégia do jogador dado a mesa atual.

Vamos implementar nossa classe base.

In [8]:
class Agent(object):
    def __call__(self, *state):
        raise NotImplementedError('You need to overwrite the __call__ method.')

Certo, já podemos testar nossa simulação com algumas estratégias. Vamos começar simulando um perdedor, um jogador que sempre "estoura" os 21.

In [9]:
class Looser(Agent):
    def __call__(self, *state):
        """This should never win, for sanity check"""
        return True

Como um teste de validade da nossa função de simulação, vamos perder mil jogos!

In [10]:
for i in range(1000):
    assert not simulate_game(Looser())
    
print('All good till here.')

All good till here.


Uma estratégia simples é sempre pedir até completar 17 pontos. Vamos tentar.

In [11]:
class Simple(Agent):
    def __call__(self, *state):
        """Always call until 17, stop otherwise"""
        if state[0] < 17:
            return True
        else:
            return False

In [12]:
simulate(Simple(), 5000)

4999/5000 - Player won 2055 out of 4999 (win rate = 41.099999999999994%)
Player won 2055 out of 5000 (win rate = 41.099999999999994%)


42% pode não parecer muito, mas essa estratégia é muito próximo do ótimo para este jogo.

## Aprendizado de Máquina

Vamos ver o que podemos fazer utilizando técnicas de *reinforcement learning*.

Por se tratar de um jogo muito simples, podemos utilizar uma técnica também simples. Vamos implementar um modelo linear nos fatores do ambiente e treinaremos esse modelo utilizando o algoritmo *REINFORCE Policy Gradients*, de Willians.

O pseudo código para esse algoritmo é o seguinte:

```
função REINFORCE
    inicialize theta arbitrário
    para cada episodio {s1, a1, r2, ..., st-1, at-1, rt} ~ pi_theta faça:
        para t = 1 até T - 1 faça
            theta <- theta + alpha.delta_theta.log_pi_theta(st, at).vt
        fim for
    fim for
    returne theta
fim função
```

Vamos implementar uma subclasse de nosso agente original para utilizar em nosso ambiente simulado.

In [95]:
class REINFORCE(Agent):
    '''
    REINFORCE Policy Gradients agent with softmax shallow model
    https://homes.cs.washington.edu/~todorov/courses/amath579/reading/PolicyGradient.pdf
    https://www.youtube.com/watch?v=2pWv7GOvuf0&list=PLqYmG7hTraZDM-OYHWgPebj2MfCFzFObQ
    https://medium.com/samkirkiles/reinforce-policy-gradients-from-scratch-in-numpy-6a09ae0dfe12
    '''
    def __init__(self, state_dim, n_actions, learning_rate, gamma, train=False):
        ''' Holds our model weights and hyper parameters '''
        self._train = train
        self.w = np.random.rand(state_dim, n_actions) * 0.1
        self.n_actions = n_actions
        self.lr = learning_rate
        self.g = gamma
        self.grads = []
        self.rewards = []
    
    def __call__(self, *state):
        if self._train:
            return self.train(*state)
        else:
            return self.predict(*state)
            
    @staticmethod
    def preprocess_state(state):
        ''' Center and scale environment variables '''
        return np.array([
            state[0] / 21 - 0.5,
            state[1] / 21 - 0.5,
            state[2] / 4 - 0.5
        ]).reshape((1, -1))
    
    @staticmethod
    def softmax_grad(softmax):
        ''' Vectorized softmax Jacobian '''
        s = softmax.reshape(-1,1)
        return np.diagflat(s) - np.dot(s, s.T)
    
    def policy(self, state):
        ''' Our policy that maps state to action parameterized by w '''
        exp = np.exp(state.dot(self.w))
        probs = exp / np.sum(exp)
        action = np.random.choice(self.n_actions, p=probs[0])
        return action, probs
    
    def predict(self, *state):
        '''
        Perform model inference and select
        the best action greedly
        '''
        state = self.preprocess_state(state)
        return np.argmax(self.policy(state)[1][0])
        
    def train(self, *state):
        '''
        Perform model inference, select action
        and store gradients for training
        '''
        state = self.preprocess_state(state)
        action, probs = self.policy(state)
        dsoftmax = self.softmax_grad(probs)[action,:]
        dlog = dsoftmax / probs[0, action]
        grad = state.T.dot(dlog[None,:])
        self.grads.append(grad)
        self.rewards.append(0)
        return action

    def update(self, reward):
        '''
        Use the returned reward to compute gradients
        and update model parameters
        '''
        self.rewards[-1] = reward
    
        for i in range(len(self.grads)):
            # Loop through everything that happend in the episode and update towards the log policy gradient times **FUTURE** reward
            self.w += self.lr * self.grads[i] * sum([r * (self.g ** r) for t, r in enumerate(self.rewards[i:])])
        
        self.grads = []
        self.rewards = []

Ainda temos que escrever uma rotina de treinamento para o nosso modelo.

Simularemos o resultado de vários jogos e, para cada jogo, atualizaremos nosso modelo com base no resultado de suas ações. Queremos utilizar como sinal de treinamento valores entre -1 e 1, sendo -1 reforço negativo e 1 reforço positivo.

In [93]:
def train(agent, n_games=5):
    player_victories = 0
    for i in range(n_games):
        reward = simulate_game(agent)
        player_victories += reward
        agent.update(reward * 2 - 1)
        
        if (i + 1) % 1000 == 0:
            print(f'{i}/{n_games} - Player won {player_victories} out of {i} (win rate = {player_victories / (i + 1) * 100}%)', end='\r')
        
    print(f'\nPlayer won {player_victories} out of {n_games} (win rate = {player_victories / n_games * 100}%)')

In [94]:
reinforce = REINFORCE(3, 2, 0.01, 0.999, train=True)
train(reinforce, 500000)

Player won 202905 out of 500000 (win rate = 40.581%) rate = 40.581%)595190381%))


Com o agente já treinado, podemos avaliar seu desempenho.

In [91]:
reinforce._train = False
simulate(reinforce, 50000)

49999/50000 - Player won 20799 out of 49999 (win rate = 41.598%)5306122446%)
Player won 20799 out of 50000 (win rate = 41.598%)


Nosso modelo aprende!

O resultado ficou dentro do esperado, em aproximadamente 42% vitórias. Esse jogo é desenhado para ser vantajoso para a banca, não importa o quão inteligente se torne o nosso jogador.

## Referências
- [Intro to Reinforcement Learning - David Silver](https://www.youtube.com/watch?v=2pWv7GOvuf0&list=PLqYmG7hTraZDM-OYHWgPebj2MfCFzFObQ)

- [Advanced Deep Learning & Reinforcement Learning](https://www.youtube.com/watch?v=iOh7QUZGyiU&list=PLqYmG7hTraZDNJre23vqCGIVpfZ_K2RZs)

- [CS885 Reinforcement Learning - University of Waterloo](https://www.youtube.com/watch?v=xoxz-OmcL1Q&list=PLdAoL1zKcqTXFJniO3Tqqn6xMBBL07EDc)


## Apêndice

A derivada da função *softmax* é a seguinte:

Primeiro, note que: $$\pi_\theta(s,a) = softmax = \frac{e^{\phi(s,a)^\intercal\theta}}{\sum_{k=1}^Ne^{\phi(s,a_k)^\intercal\theta}}$$

Utilizando  $\log$ identity $\log(x/y) = \log(x) - \log(y)$ podemos escrever $$\log(\pi_\theta(s,a)) = \log(e^{\phi(s,a)^\intercal\theta}) - \log(\sum_{k=1}^Ne^{\phi(s,a_k)^\intercal\theta}) $$

Agora, tomamos o gradiente:

$$\nabla_\theta\log(\pi_\theta(s,a)) = \nabla_\theta\log(e^{\phi(s,a)^\intercal\theta}) - \nabla_\theta\log(\sum_{k=1}^Ne^{\phi(s,a_k)^\intercal\theta})$$

O lado esquerdo da equação é simplificado como:

$$left= \nabla_\theta\log(e^{\phi(s,a)^\intercal\theta}) = \nabla_\theta\phi(s,a)^\intercal\theta = \phi(s,a)$$

O lado direito da equação simplifica para:

Utilizando a regra da cadeia: $$\nabla_x\log(f(x)) = \frac{\nabla_xf(x)}{f(x)}$$

Temos:

$$right = \nabla_\theta\log(\sum_{k=1}^Ne^{\phi(s,a_k)^\intercal\theta}) = \frac{\nabla_\theta\sum_{k=1}^Ne^{\phi(s,a_k)^\intercal\theta}}{\sum_{k=1}^Ne^{\phi(s,a_k)^\intercal\theta}}$$

Tomando o gradiente do numerador, temos:

$$right = \frac{\sum_{k=1}^N{\phi(s,a_k)}e^{\phi(s,a_k)^\intercal\theta}}{\sum_{k=1}^Ne^{\phi(s,a_k)^\intercal\theta}}$$

Substituindo a definição de $\pi_\theta(s,a)$ podemos simplificar para:

$$right = \sum_{k=1}^N{\phi(s,a_k)}\pi_\theta(s,a_k)$$

Dada a definição de valor esperado, temos:

$$\mathrm{E}[X] = X \cdot P = x_1p_1+x_2p_2+ ... +x_np_n$$

O que em português traduz para a soma de cada fator vezes sua probabilidade.

$$X = features = {\phi(s,a)}$$

$$P = probabilities =\pi_\theta(s,a)$$

Assim, temos o valor esperado de cada fator:

$$right = \mathrm{E}_{\pi_\theta}[\phi(s,\cdot)]$$

onde $\cdot$ significa todas as ações possíveis.

Juntando tudo: $$\nabla_\theta\log(\pi_\theta(s,a)) = left - right = \phi(s,a) - \mathrm{E}_{\pi_\theta}[\phi(s,\cdot)]$$